In [161]:
from os import walk
from os.path import join
import pandas as pd

In [162]:
spam_1_path = 'SpamData/01_Processing/spam_assassin_corpus/spam_1'
spam_2_path = 'SpamData/01_Processing/spam_assassin_corpus/spam_2'
nonspam_1_path = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_1'
nonspam_2_path = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_2'
spam_cat =1 
ham_cat =0

In [163]:
#Reading files
eg_file = 'SpamData/01_Processing/practice_email.txt'
stream=open(eg_file,encoding="latin-1")
message=stream.read()
stream.close()

In [164]:
stream=open(eg_file)
is_body=False
lines=[]
for line in stream:
    if is_body:
        lines.append(line)
    elif line == '\n':
        is_body = True
stream.close()
email_body = '\n'.join(lines)
print(email_body)



Dear Mr Still



Good tidings to you and all your staff for the festive season ahead (Christmas).

Now to the crux of the matter-in-hand: I am a fully qualified Santa Claus and am wondering whether you might consider me to run my own "Santa's Grotto" in your store.

But WAIT! You're probably thinking: "What makes him so special?"

Well, first of all, I have made several changes to the characterisation of Father Christmas. Rather than greeting the children with shouts of "Ho, ho, ho!" I prefer to whisper the phrase "Dependence is not unfathomable in this cruel world we live in". In addition, my gifts are ALL hand-made, ranging from felt hoops to vanilla-pod holders.

You will note also, from the enclosed sketch, that I have radically redesigned Santa's outfit and have renamed my character "Lord Buckles". Would you be interested in employing me? I promise NEVER to let you down.

I look forward to hearing from you.



Best wishes

Robin Cooper

[Excerpt from the book: The Timewaster Let

# Creating Pandas DataFrame of email bodies

In [165]:
def email_body_generator(path):
    for root,dirnames,filenames in walk(path):
        for file_name in filenames:
            filepath = join(root,file_name)
            stream = open(filepath, encoding="latin-1")
            is_body = False
            lines = []
            for lines in stream:
                if is_body:
                    lines.append(line)
                elif line == '\n':
                    is_body = True
                email_body='\n'.join(lines)
                yield file_name,email_body
stream.close()

In [166]:
def df_from_directory(path,classification):
    rows=[]
    rows_names=[]
    for file_names,email_body in email_body_generator(path):
        rows.append({'MESSAGE':email_body, 'CATEGORY':classification})
        rows_names.append(file_names)
    return pd.DataFrame(rows,index=rows_names)

In [174]:
#for spam emails
spam_emails = df_from_directory(spam_1_path,spam_cat)
spam_emails = pd.concat([spam_emails,df_from_directory(spam_2_path,spam_cat)])
print(spam_emails.shape)

(268601, 2)


In [173]:
#for non-spam emails
ham_emails = df_from_directory(spam_1_path,ham_cat)
ham_emails = pd.concat([ham_emails, df_from_directory(spam_2_path,ham_cat)])
print(ham_emails.shape)

(268601, 2)


In [176]:
#concating both spam and non-emails
data = pd.concat([spam_emails,ham_emails])
print('shape of the whole dataframe',data.shape)

shape of the whole dataframe (537202, 2)
